Initial attempt at saving the image from ChromeDino so that we can then try and make use of the game environment

In [ ]:
import re
import time
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, tnrange
import numpy as np
from collections import defaultdict,Counter
from scipy.misc import imread, imsave

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import base64
import io
from PIL import Image

from matplotlib import pyplot as plt, rcParams
from matplotlib import cm
%matplotlib inline
rcParams["figure.figsize"] = (16,12)    # Use this line to increase your figure size (optional).

# Option 2
Force Selenium to execute JavaScript to just pull the information out of the canvas tag.

In [ ]:
def detect_object(img):
    x,y = np.shape(img)
    dino = img[x//2:,:x//2] #prob don't need this
    close = img[x//2:,x//2+20:x+20]
    return np.mean(close)
    print("Calculated {}",np.mean(close))

js_call = "return document.getElementsByClassName('runner-canvas')[0].toDataURL('image/png').substring(21);"
url = "https://chromedino.com/"
browser = webdriver.Chrome()
try:
    browser.get(url)

    body = browser.find_element_by_id('t')
    body.send_keys(Keys.ARROW_UP)
    prev_jump = None
    while True:
        time.sleep(.25)
        b64_image = browser.execute_script(js_call)
        image = base64.b64decode(b64_image)
        buf = io.BytesIO(image)
        img = Image.open(buf).convert('LA')
        np_img = np.array(img)[:,:,0]
        new_jump = detect_object(np_img)
        if prev_jump == new_jump and new_jump > 3:
            print("I think I died")
            break
        elif new_jump > 3:
            body.send_keys(Keys.ARROW_UP)
        else:
            print("chose to not jump")
        print(new_jump)
except Exception as e:
    print("Errored out at step 1", e)
finally:
    browser.close() #make sure we close it when we are finished.